In [ ]:
import json
import os
import sys
import cv2 as cv
import numpy as np
import os
from shutil import copyfile
from pathlib import Path
from tqdm import tqdm
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [ ]:
MaineFile = '/home/roadzen/ParasWork/ConUHack2020/data/cars_train.txt'
OUT_IMG_DIR = '/home/roadzen/ParasWork/ConUHack2020/detectron_2/datasets/car/coco_train2014/'
OUT_JSON = '/home/roadzen/ParasWork/ConUHack2020/detectron_2/datasets/car/annotations/instances_train2014.json'

In [ ]:
MaineFile = '/home/roadzen/ParasWork/ConUHack2020/data/cars_test.txt'
OUT_IMG_DIR = '/home/roadzen/ParasWork/ConUHack2020/detectron_2/datasets/car/coco_val2014/'
OUT_JSON = '/home/roadzen/ParasWork/ConUHack2020/detectron_2/datasets/car/annotations/instances_val2014.json'

In [ ]:
imageLocations = None
file = open(MaineFile,'r')
imageLocations = file.readlines()
file.close()


print(len(imageLocations))

imageLocations = list(set(imageLocations))

print(len(imageLocations))

print(imageLocations[-1][:-1])

In [ ]:
TOTAL = len(imageLocations)
#TOTAL = 100

In [ ]:
class annotation:
    def __init__(self):
        self.segmentation = []
        self.area = 0
        self.iscrowd = 0
        self.ignore = 0
        self.image_id = 0
        self.bbox = []
        self.category_id = 0
        self.id = 0

In [ ]:
class imageINFO:
    def __init__(self):
        self.id = 0
        self.file_name = 0
        self.width = 0
        self.height = 0

In [ ]:
class BOX:
    height = 0
    top = 0
    width = 0
    left = 0
    
def getBox(LABELS):
    ObjectBox = BOX()
    ObjectBox.height = float(LABELS[4]) - int(LABELS[2])
    ObjectBox.width = float(LABELS[3]) - int(LABELS[1])
    ObjectBox.left = float(LABELS[1])
    ObjectBox.top = float(LABELS[2])
    return ObjectBox

In [ ]:
def getBoxinImagePixels(ObjectBox,imageHeight,imageWidth):
    ObjectBox.height = int(float(ObjectBox.height*imageHeight))
    ObjectBox.width = int(float(ObjectBox.width*imageWidth))
    ObjectBox.left = int(float(ObjectBox.left*imageWidth))
    ObjectBox.top = int(float(ObjectBox.top*imageHeight))
    return ObjectBox

def scaleBox(ObjectBox,imageHeight,imageWidth):
    ObjectBox.height = float(ObjectBox.height/imageHeight)
    ObjectBox.width = float(ObjectBox.width/imageWidth)
    ObjectBox.left = float(ObjectBox.left/imageWidth)
    ObjectBox.top = float(ObjectBox.top/imageHeight)
    return ObjectBox

def listAllFourCoordinatesofBox(ObjectBox):
    return [int(ObjectBox.left),int(ObjectBox.top),
    int(ObjectBox.left + ObjectBox.width),int(ObjectBox.top),
    int(ObjectBox.left + ObjectBox.width),int(ObjectBox.top + ObjectBox.height),
    int(ObjectBox.left),int(ObjectBox.top + ObjectBox.height)]

In [ ]:
class OUTPUT_JSON:
    def __init__(self):
        self.type = 'instances'
        self.images = []
        self.annotations = []
        self.categories = []
    def reInit(self):
        self.images = []
        self.annotations = []
        self.categories = []
FinalJSON = OUTPUT_JSON()

In [ ]:
def getAnnotationObj(imageOBJ,INSTANCE_ID_COUNT,ObjectBox):
    annotationOBJ = annotation()
    annotationOBJ.id = INSTANCE_ID_COUNT
    annotationOBJ.image_id = imageOBJ.id
    annotationOBJ.bbox = [int(ObjectBox.left),
                          int(ObjectBox.top),
                          int(ObjectBox.width),
                          int(ObjectBox.height)]
    annotationOBJ.area = int(ObjectBox.height)*int(ObjectBox.width)
    annotationOBJ.segmentation = [listAllFourCoordinatesofBox(ObjectBox)]
    return annotationOBJ

In [ ]:
FINAL_IMAGE_HEIGHT = 448
FINAL_IMAGE_WIDTH = 448
USE_FIXED_SCALE = True

In [ ]:
IMAGE_ID_COUNT = 1
INSTANCE_ID_COUNT = 1

FinalJSON.reInit()

for index in tqdm(range(TOTAL)):
    original_IMG_PATH = imageLocations[index][:-1]
    GT_FILE = original_IMG_PATH[:-4] + '.txt'
    
    lines = 0
    with open(GT_FILE) as textFile:
        lines = [line.split() for line in textFile]
    
    image = cv.imread(original_IMG_PATH)
    imageHeight,imageWidth,_ = image.shape
    
    imageOBJ = imageINFO()
    imageOBJ.id = IMAGE_ID_COUNT
    imageOBJ.height,imageOBJ.width = (FINAL_IMAGE_HEIGHT, FINAL_IMAGE_WIDTH)
    imageOBJ.file_name = str(IMAGE_ID_COUNT)
    imageOBJ.file_name = imageOBJ.file_name.zfill(6) + '.jpg'
    outputFilePath = OUT_IMG_DIR + imageOBJ.file_name
    FinalJSON.images.append(vars(imageOBJ))
    
    for line in lines:
        label = line[0]
        ObjectBox = getBox(line)
        ObjectBox = scaleBox(ObjectBox,imageHeight,imageWidth)
        ObjectBox = getBoxinImagePixels(ObjectBox,FINAL_IMAGE_HEIGHT,FINAL_IMAGE_WIDTH)
        annotationOBJ = getAnnotationObj(imageOBJ,INSTANCE_ID_COUNT,ObjectBox)
        INSTANCE_ID_COUNT += 1
        annotationOBJ.category_id = int(label)
        FinalJSON.annotations.append(vars(annotationOBJ)) 
    if USE_FIXED_SCALE:
        image = cv.resize(image, (FINAL_IMAGE_HEIGHT, FINAL_IMAGE_WIDTH))
    cv.imwrite(outputFilePath,image)
    IMAGE_ID_COUNT += 1

In [ ]:
from serverCode.classes import *

In [ ]:
class category:
    def __init__(self):
        self.supercategory = 'none'
        self.id = -1
        self.name = ''
for index in range(len(CLASSES)):
    categoryOBJ = category()
    categoryOBJ.name = CLASSES[index]
    categoryOBJ.id = index + 1
    FinalJSON.categories.append(vars(categoryOBJ))

In [ ]:
import json

with open(OUT_JSON, 'w') as fp:
    json.dump(vars(FinalJSON), fp)